In [1]:
import numpy as np
from river import datasets
from river import evaluate
from river import linear_model
from river import metrics
from river import optim
from river import preprocessing
from river import drift
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
# Load dataset
dataset = datasets.synth.Agrawal(
     classification_function=0,
     seed=42
)

# Load pickled model
LRmodel1 = pickle.load(open('../saved_models/classification/sklearn/LogRegsk.pkl', 'rb'))

# Initialize drift detector
drift_detector = drift.ADWIN(delta=1.2)
drifts = list()

# Initialise metric
metric = metrics.Accuracy()

# buffers to store data stream for re-trainig model
feature_buffer = list()
label_buffer = list()

for i,data in enumerate(dataset.take(1000)):
    # data[0] - features, data[1] - label
    y_pred = LRmodel1.predict(np.reshape(list(data[0].values()), (-1, 1)).T)
    metric = metric.update(data[1],y_pred[0])
    correctly_classified = y_pred == data[1]
    drift_detector.update(correctly_classified)   # Data is processed one sample at a time
    feature_buffer.append(list(data[0].values()))
    label_buffer.append(data[1])
    if drift_detector.drift_detected:
        # The drift detector indicates after each sample if there is a drift in the data
        print(f'Change detected at index {i}')
        drifts.append(i)
        print(f'Re-train the model')
        LRmodel1.fit(feature_buffer,label_buffer)
        print(LRmodel1)

print(metric)

Change detected at index 95
Re-train the model
Pipeline(steps=[('scaler', StandardScaler()), ('logreg', LogisticRegression())])
Change detected at index 319
Re-train the model
Pipeline(steps=[('scaler', StandardScaler()), ('logreg', LogisticRegression())])
Accuracy: 65.20%


/Users/sebasmos/anaconda3/envs/book/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/sebasmos/anaconda3/envs/book/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LogisticRegression from version 1.3.2 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/sebasmos/anaconda3/envs/book/lib/python3.10/site-packages/sklearn/base.py:347: InconsistentVersionWarning: Trying to unpickle estimator Pipeline from ver